In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import hashlib
from datetime import datetime

In [ ]:
# ============ DCGAN Architecture ============
class Generator(nn.Module):
    def __init__(self, nz=100, ngf=64, nc=3):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Input: (nz) x 1 x 1
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # State: (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # State: (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # State: (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # State: (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output: (nc) x 64 x 64
        )

    def forward(self, x):
        return self.main(x)


class Discriminator(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Input: (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # State: (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x).view(-1, 1).squeeze(1)

In [ ]:
# ============ 1D Exponential Chebyshev Chaotic Map ============
class ChaoticKeyGenerator:
    def __init__(self, initial_seed=0.123456789, r=3.99):
        """
        1D Exponential Chebyshev Map for key generation
        x_{n+1} = exp(r * x_n * (1 - x_n))
        """
        self.x = initial_seed
        self.r = r

    def generate_sequence(self, length):
        """Generate chaotic sequence of given length"""
        sequence = []
        for _ in range(length):
            # 1D-EC map iteration
            self.x = np.exp(self.r * self.x * (1 - self.x)) % 1.0
            sequence.append(self.x)
        return np.array(sequence)

    def generate_key_sequences(self, num_sequences=8, seq_length=256):
        """Generate multiple chaotic key sequences"""
        keys = []
        for i in range(num_sequences):
            # Use different initial conditions for each sequence
            self.x = (self.x + 0.1 * i) % 1.0
            key_seq = self.generate_sequence(seq_length)
            keys.append(key_seq)
        return keys

In [ ]:
# ============ DCGAN-Based Key Generator ============
class DCGANKeyGenerator:
    def __init__(self, nz=100, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.nz = nz
        self.generator = Generator(nz=nz).to(device)
        self.chaotic_gen = ChaoticKeyGenerator()

    def load_pretrained(self, path):
        """Load pre-trained DCGAN generator"""
        self.generator.load_state_dict(torch.load(path, map_location=self.device))
        self.generator.eval()

    def generate_key_image(self, noise=None):
        """Generate synthetic image using DCGAN"""
        if noise is None:
            noise = torch.randn(1, self.nz, 1, 1, device=self.device)

        with torch.no_grad():
            fake_image = self.generator(noise)

        return fake_image

    def extract_base_key(self, image_tensor, timestamp=None, nonce=None):
        """
        Extract base key from generated image using SHA-512
        Combines pixel data, timestamp, and nonce
        """
        # Convert image to numpy and calculate pixel sums
        img_np = image_tensor.cpu().numpy().squeeze()
        pixel_sum = np.sum(img_np, axis=(0, 1, 2))

        # Add timestamp
        if timestamp is None:
            timestamp = datetime.now().timestamp()

        # Add random nonce
        if nonce is None:
            nonce = np.random.randint(0, 2**32)

        # Combine all components
        key_data = f"{pixel_sum}_{timestamp}_{nonce}".encode()

        # Generate SHA-512 hash (512 bits = 64 bytes)
        base_key = hashlib.sha512(key_data).digest()

        return base_key, timestamp, nonce

    def generate_encryption_keys(self, message_length=256):
        """
        Complete key generation pipeline:
        1. Generate DCGAN image
        2. Extract base key using SHA-512
        3. Use chaotic map to generate 8 key sequences
        """
        # Step 1: Generate image
        noise = torch.randn(1, self.nz, 1, 1, device=self.device)
        key_image = self.generate_key_image(noise)

        # Step 2: Extract base key
        base_key, timestamp, nonce = self.extract_base_key(key_image)

        # Step 3: Initialize chaotic generator with base key
        seed = int.from_bytes(base_key[:8], byteorder='big') / (2**64)
        self.chaotic_gen.x = seed

        # Step 4: Generate 8 chaotic key sequences
        key_sequences = self.chaotic_gen.generate_key_sequences(
            num_sequences=8,
            seq_length=message_length
        )

        return {
            'base_key': base_key,
            'key_sequences': key_sequences,
            'timestamp': timestamp,
            'nonce': nonce,
            'key_image': key_image,
            'noise': noise
        }

In [ ]:
# ============ DCGAN Training Function ============
def train_dcgan(dataloader, num_epochs=50, nz=100, lr=0.0002, beta1=0.5):
    """
    Train DCGAN on image dataset
    dataloader: PyTorch DataLoader with images
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    netG = Generator(nz=nz).to(device)
    netD = Discriminator().to(device)

    # Initialize weights
    def weights_init(m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)

    netG.apply(weights_init)
    netD.apply(weights_init)

    criterion = nn.BCELoss()
    optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

    fixed_noise = torch.randn(64, nz, 1, 1, device=device)

    print("Starting DCGAN Training...")

    for epoch in range(num_epochs):
        for i, data in enumerate(dataloader):
            # Update Discriminator
            netD.zero_grad()
            real_images = data[0].to(device)
            batch_size = real_images.size(0)
            labels = torch.full((batch_size,), 1.0, device=device)

            output = netD(real_images)
            errD_real = criterion(output, labels)
            errD_real.backward()

            noise = torch.randn(batch_size, nz, 1, 1, device=device)
            fake_images = netG(noise)
            labels.fill_(0.0)

            output = netD(fake_images.detach())
            errD_fake = criterion(output, labels)
            errD_fake.backward()

            errD = errD_real + errD_fake
            optimizerD.step()

            # Update Generator
            netG.zero_grad()
            labels.fill_(1.0)
            output = netD(fake_images)
            errG = criterion(output, labels)
            errG.backward()
            optimizerG.step()

            if i % 50 == 0:
                print(f'[{epoch}/{num_epochs}][{i}/{len(dataloader)}] '
                      f'Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}')

        # Save checkpoint
        if (epoch + 1) % 10 == 0:
            torch.save(netG.state_dict(), f'generator_epoch_{epoch+1}.pth')

    return netG, netD

In [ ]:
# ============ Usage Example ============
if __name__ == "__main__":
    # Initialize key generator
    key_gen = DCGANKeyGenerator()

    # For testing without pre-trained model (random weights)
    print("Generating encryption keys...")
    keys = key_gen.generate_encryption_keys(message_length=256)

    print(f"Base key size: {len(keys['base_key'])} bytes")
    print(f"Number of key sequences: {len(keys['key_sequences'])}")
    print(f"Each sequence length: {len(keys['key_sequences'][0])}")
    print(f"Timestamp: {keys['timestamp']}")
    print(f"Nonce: {keys['nonce']}")
    print(f"Key space: 2^440 (approximately)")

Generating encryption keys...
Base key size: 64 bytes
Number of key sequences: 8
Each sequence length: 256
Timestamp: 1759412263.25324
Nonce: 1745586709
Key space: 2^440 (approximately)
